In [8]:
import random
import numpy as np

import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer

In [ ]:
class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def step(self):
        # The agent's step will go here.
        # For demonstration purposes we will print the agent's unique_id
        print("Hi, I am agent " + str(self.unique_id) + ".")

The agents ($i$'s):

Problem_solving = $S_i$ # from 0 to 1$

Cps_level = $L_i$ 
    - if problem solving is 1.92 to 2 then cps level is 4 
    - if problem solving is 1.64 to 1.92 then cps level is 3
    - if problem solving is 1.29 to 1.64 then cps level is 2
    - if problem solving is 1 to 1.29 then cps level is 1 

Communication = $C_i$# from 1 to 2 (!for now)

Grid / The Tasks:
- Task_Complexity = From 0 to 1

The AgentStep:
At each tick the agent evaluates the current problem solving ability of all agents on the grid and either 1. moves, 2. communicates or work
1. Move
- If *i* !working then move_to() nearby grid

2. Evaluate
- If $S_i$ > Task_Complexity then work individually
- If $S_i$ <= Task_Complexity then suggest Collaboration

3. Communicate / Collaborate
- Communicate with a nearby agent in the "8" squares around (!For now - look in communication litetarture)
    - Communication : Succes = 50% (!For now - comm literature) * $C_i$
    - If Communication == Success then intiate collaboration discussion
    31. Collab Discussion
    - Agent $i$ sends to $i_R$(Receiver)
    - if $i_R$ !working then $i_R$ jumps to task (!for now - maybe add a propability, such that a person with low $S_i$ might not join task, to reflect natural errors)
    - if $i_R$ :working then:
        - $i$ joins grid_cell if $S_i$ + $S_{i_R}$ > Task_Complexity

4. Work
- If problem solving ability of all agents is larger than Task complexity then begin working
- One tick is used on working instead of moving or communicating








In [15]:
class TeamAgent(mesa.Agent):
    """An agent in a team with a range of teamwork skill parameters"""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.problem_solving = problem_solving # 1 to 2 # how good they are at solving problems
        self.cps_level = getcps_level(problem_solving) # 1 to 4 # how good they are at solving problems
        self.communication =  random.random() + 0.5 # 0.5 to 1.5 # how good they are at communicating
        self.working = False # By default the will not work on a task
        self.pos
    
    def get_cps_level(problem_solving_value):
        if problem_solving_value >= 1.92 and problem_solving_value <= 2:
            return 4
        elif problem_solving_value >= 1.64 and problem_solving_value < 1.92:
            return 3
        elif problem_solving_value >= 1.29 and problem_solving_value < 1.64:
            return 2
        elif problem_solving_value >= 1 and problem_solving_value < 1.29:
            return 1
        else:
            return None
    
    # A move function which will be used for the step function later on
    def move(self):
        if not self.working: # if they are not working on a task
            possible_moves = self.model.grid.get_neighborhood(  # retrieve neighborhood grids
                self.pos, moore=True, include_center=False 
            )
            new_position = random.choice(possible_moves) #
            self.model.grid.move_agent(self, new_position)

    def grid_problem_solving(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos]) # Retrieves a a list of all agents in the same cell
        self.team_problem_solving = 0
        if len(cellmates) > 1:
            for mate in cellmates:
                self.team_problem_solving += mate.problem_solving

    def evaluate(self):
        if model.grid.accomplished == True:
            self.working = False
        if self.problem_solving > self.model.task_complexity:
            self.work_individually()
        elif self.team_problem_solving > self.model.task_complexity:
            self.work_as_a_team()
        else:
            self.suggest_collaboration()
        
    def work_individually(self):
        self.working = True
        self.work()

    def work_as_a_team(self):
        self.working = True
        self.work()

    def suggest_collaboration(self):
        neighbors = self.model.grid.get_neighbors(
            self.pos, moore=True, include_center=False
        )
        for neighbor in neighbors:
            if not neighbor.working and ( neighbor.problem_solving + self.problem_solving ) > self.model.task_complexity:
                neighbor.model.grid.move_agent(neighbor, self.pos)

    def work(self):
        if self.working:
            x, y = self.pos
            self.model.space.accomplished[x][y] = True
            
        
    
  #      for neighbor in self.model.grid.neighbor_iter(self.pos):
  #          if neighbor.problem_solving + self.problem_solving > task_complexity:
  #              similar += 1

    def step(self):
        self.evaluate()
        self.move()
        


In [30]:
# Custom grid build

class CustomGrid(MultiGrid):
    def __init__(self, width, height):
        super().__init__(width, height, torus=True)
        self.task_complexity = self.build_grid()
        self.accomplished = self.build_grid()

    def build_grid(self):
        grid = np.empty((self.width, self.height), dtype=object)
        for x, y in self.coord_iter():
            grid[x][y] = {
                'task_complexity': np.random.uniform(0, 2),
                'accomplished': False
            }
        return grid

In [31]:

# Create the model
class TeamworkModel(mesa.Model):
    """ A model with some number of agents."""
    
    def __init__(self, num_agents, width, height):
        self.num_agents = num_agents
        self.grid = self.grid = CustomGrid(width, height)
        self.schedule = RandomActivation(self)



        # Create agents
        for i in range(self.num_agents):
            problem_solving = random.uniform(0, 2)
            communication = random.uniform(1, 2)
            agent = TeamworkAgent(i, self, problem_solving, communication)
            self.schedule.add(agent)
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

    def access_grid_values(self):
            for cell in self.grid.coord_iter():
                x, y = cell
                value = self.grid.task_complexity[x][y]
                accomplished = self.grid.accomplished[x][y]
                print(f"Cell ({x}, {y}) - Task Complexity: {value}, Accomplished: {accomplished}")


    def step(self):
        self.schedule.step()


In [32]:
# Create the model and run the simulation
model = TeamworkModel(num_agents=10, width=10, height=10)
for _ in range(100):
    model.step()

# Access the agent positions
for agent in model.schedule.agents:
    print(agent.pos)

ValueError: too many values to unpack (expected 2)

In [9]:
class TeamworkAgent(mesa.Agent):
    def __init__(self, unique_id, model, problem_solving, communication):
        super().__init__(unique_id, model)
        self.problem_solving = problem_solving # how good they are at solving problems
        self.communication = communication # how good they are at communicating
        self.working = False # whether they are working on a task

    def move(self):
        if not self.working: # if they are not working on a task
            possible_moves = self.model.grid.get_neighborhood( 
                self.pos, moore=True, include_center=False 
            )
            new_position = random.choice(possible_moves) #
            self.model.grid.move_agent(self, new_position)

    def evaluate(self):
        if self.problem_solving > self.model.task_complexity:
            self.work_individually()
        else:
            self.suggest_collaboration()

    def suggest_collaboration(self):
        neighbors = self.model.grid.get_neighbors(
            self.pos, moore=True, include_center=False
        )
        for neighbor in neighbors:
            if not neighbor.working:
                neighbor.jump_to_task()
                break

    def jump_to_task(self):
        self.working = True

    def collaborate(self, receiver):
        if not receiver.working:
            receiver.jump_to_task()
        elif self.problem_solving + receiver.problem_solving > self.model.task_complexity:
            self.jump_to_task()

    def work_individually(self):
        self.working = True

    def work(self):
        # Simulate work progress
        pass

    def step(self):
        self.move()
        self.evaluate()
        self.work()


In [11]:

# Create the model
class TeamworkModel(mesa.Model):
    """ A model with some number of agents."""
    
    def __init__(self, num_agents, task_complexity):
        self.num_agents = num_agents
        self.task_complexity = task_complexity
        self.grid = mesa.space.MultiGrid(10, 10, torus=True)
        self.schedule = RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            problem_solving = random.uniform(0, 2)
            communication = random.uniform(1, 2)
            agent = TeamworkAgent(i, self, problem_solving, communication)
            self.schedule.add(agent)
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

    def step(self):
        self.schedule.step()

(7, 3)
(1, 4)
(2, 7)
(9, 5)
(7, 8)
(0, 8)
(0, 4)
(7, 0)
(6, 1)
(6, 7)


In [51]:
# Create a CanvasGrid visualization element
def agent_portrayal(agent):
    portrayal = {
        "Shape": "circle",
        "Color": "red" if agent.working else "blue",
        "Filled": "true",
        "Layer": 0,
        "r": 0.5
    }
    return portrayal

# Create the model and run the simulation
model = TeamworkModel(num_agents=10, task_complexity=0.5)

# Create the grid visualization
grid = mesa.visualization.CanvasGrid(agent_portrayal, 10, 10, 500, 500)

# Create the server to visualize the model
server = mesa.visualization.ModularServer(
    TeamworkModel,
    [grid],
    "Teamwork Model",
    {"num_agents": 10, "task_complexity": 0.5}
)

server.port = 8521  # Set the port for the web server
server.launch()

Interface starting at http://127.0.0.1:8521


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
